In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import xarray as xr
import cartopy.crs as ccrs
from pyresample.geometry import AreaDefinition
from pyresample.geometry import GridDefinition
from pyresample import image, geometry, load_area, save_quicklook, SwathDefinition, area_def2basemap
from pyresample.kd_tree import resample_nearest
from scipy import spatial
sys.path.append('../saildrone/subroutines/')
from read_routines import read_all_usv,read_one_usv, add_coll_vars_ds_rss, add_coll_vars_ds_jpl,get_filelist_l2p,get_orbital_data_l2p
import warnings
warnings.simplefilter('ignore') # filter some warning messages
from glob import glob

ds = xr.open_dataset('f:/data/sat_data/distance_to_land_25km.nc').rename({'i2':'lon','j2':'lat'})
ds['lat'],ds['lon']=np.arange(-89.875,89.876,.25),np.arange(-179.875,179.876,.25)#np.arange(.125,359.876,.25)
#ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
#ds = ds.sortby(ds.lon)
ds_land=ds
plt.rcParams['figure.figsize'] = (15,5)
ds_land.dist_land.plot()

# Read in All Saildrone cruises downloaded from https://data.saildrone.com/data/sets
- 2017 onwards, note that earlier data is going to lack insruments and be poorer data quality in general
- For this code I want to develop a routine that reads in all the different datasets and creates a standardized set
- It may work best to first read each of the files individually into a dictionary 
- then go through each dataset finding all variable names
- I decided to put all SST into TEMP_CTD_MEAN and same for Salinity so there is a single variable name
- this still preserves all the dataset information

In [ ]:
dir_data = 'C:/Users/gentemann/Google Drive/public/ALL_Saildrone_Data/' #'f:/data/cruise_data/saildrone/saildrone_data/'
dir_data_pattern = 'C:/Users/gentemann/Google Drive/public/ALL_Saildrone_Data/*.nc' 
#dir_data 'f:/data/cruise_data/saildrone/saildrone_data/'
#dir_data_pattern = 'f:/data/cruise_data/saildrone/saildrone_data/*.nc'

#get list of all filenames in directory
files = [x for x in glob(dir_data_pattern)]
print('number of file:',len(files))
for ifile,file in enumerate(files):
    print(ifile,file)


## An example showing how the using matplotlib maps orbital data quickly and easily


In [ ]:
#ds_tem = xr.open_dataset('C:/Users/gentemann/Google Drive/public/2019_saildrone/saildrone_arctic_sd1037_2019.nc')
#ds_tem2 = xr.open_dataset('f:/data/cruise_data/saildrone/2019_arctic/post_mission/saildrone-gen_5-arctic_misst_2019-sd1037-20190514T230000-20191011T183000-1_minutes-v1.1575487464625.nc')
#ds_tem = ds_tem.isel(row=slice(60,-1))
#ds_tem2 = ds_tem2.isel(obs=slice(60*24,-1))
#print(ds_tem.time[0].data,ds_tem.time[-1].data)
#print(ds_tem2.time[0,0].data,ds_tem2.time[0,-1].data)
#ds
#ds_usv

In [ ]:
adir = 'C:/Users/gentemann/Google Drive/public/ALL_Saildrone_Data/'
ds_usv = xr.open_dataset(files[4]).rename({'latitude':'lat','longitude':'lon'})
#file = 'F:/data/sat_data/smap/SSS/L2/JPL/V4.3/2016/002/SMAP_L2B_SSS_04909_20160102T044855_R16010_V4.3.h5'
file = 'F:/data/sat_data/smap/SSS/L2/RSS/V3/40km/2018/115/RSS_SMAP_SSS_L2C_40km_r17250_20180425T004136_2018115_FNL_V03.0.nc'
ds = xr.open_dataset(file)
ds.close()
x = ds.cellon.data
y = ds.cellat.data
z = ds.sss_smap.data
ax = plt.axes(projection=ccrs.PlateCarree())
cs1 = ax.scatter(x, y, s=1.0, c=z, edgecolor='none', cmap='jet')
minlon,maxlon,minlat,maxlat = ds_usv.lon.min().data,ds_usv.lon.max().data,ds_usv.lat.min().data,ds_usv.lat.max().data
ax.plot([minlon,maxlon,maxlon,minlon,minlon],[minlat,minlat,maxlat,maxlat,minlat])
#ax.plot(ds.cellon[jj,ii],ds.cellat[jj,ii,0],'b*')
#ax.plot(ds_usv.lon[1000],ds_usv.lat[1000],'ro')
ax.plot(ds_usv.lon,ds_usv.lat,'ro')
ax.coastlines()
ax.set_xlim(-130,-110)
ax.set_ylim(25,40)

# test read in a file to look at dimension names

In [ ]:
#file = 'F:/data/sat_data/smap/SSS/L2/JPL/V4.3/2016/002/SMAP_L2B_SSS_04909_20160102T044855_R16010_V4.3.h5'
file = 'F:/data/sat_data/smap/SSS/L2/RSS/V4/SCI/2018/115/RSS_SMAP_SSS_L2C_r17250_20180425T004136_2018115_FNL_V04.0.nc'
ds = xr.open_dataset(file,decode_cf=False)
ds.close()

ii=0
plt.rcParams['figure.figsize'] = (15.0,6.0)
ds2 = ds.where((ds.cellon>-180)&(ds.sss_smap>0))
ax = plt.subplot(221,projection=ccrs.PlateCarree(central_longitude=0))
cs1=ax.scatter(ds2.cellon[:,:,ii],ds2.cellat[:,:,ii],c=ds2.sss_smap[:,:,ii],vmin=30,vmax=36,edgecolor='none', cmap='jet')
ax.coastlines()
cax = plt.colorbar(cs1,shrink=.3)
ax = plt.subplot(222,projection=ccrs.PlateCarree(central_longitude=0))
cs1=ax.scatter(ds2.cellon[:,:,ii],ds2.cellat[:,:,ii],c=ds2.iqc_flag[:,:,ii],vmin=0,vmax=32000,edgecolor='none', cmap='jet')
ax.coastlines()
cax = plt.colorbar(cs1,shrink=.3)
ii=1
ds2 = ds.where((ds.cellon>-180)&(ds.sss_smap>0))
ax = plt.subplot(223,projection=ccrs.PlateCarree(central_longitude=0))
cs1=ax.scatter(ds2.cellon[:,:,ii],ds2.cellat[:,:,ii],c=ds2.sss_smap[:,:,ii],vmin=30,vmax=36,edgecolor='none', cmap='jet')
ax.coastlines()
cax = plt.colorbar(cs1,shrink=.3)
ax = plt.subplot(224,projection=ccrs.PlateCarree(central_longitude=0))
cs1=ax.scatter(ds2.cellon[:,:,ii],ds2.cellat[:,:,ii],c=ds2.iqc_flag[:,:,ii],vmin=0,vmax=32000,edgecolor='none', cmap='jet')
ax.coastlines()
cax = plt.colorbar(cs1,shrink=.3)
#print(ds)
fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/SSS/figures/RSS_r17250.png'
plt.savefig(fig_fname, transparent=False, format='png')

In [ ]:
file = 'F:/data/sat_data/smap/SSS/L2/JPL/V5.0/2018/115/SMAP_L2B_SSS_17250_20180425T004224_R17000_V5.0.h5'
ds = xr.open_dataset(file,decode_cf=False)
ds.close()
plt.rcParams['figure.figsize'] = (15.0,10.0)
ds2 = ds.where((ds.lon>-180)&(ds.smap_sss>0))
ax = plt.subplot(121,projection=ccrs.PlateCarree(central_longitude=0))
cs1=ax.scatter(ds2.lon,ds2.lat,c=ds2.smap_sss,vmin=30,vmax=36,edgecolor='none', cmap='jet')
#cs1=ax.scatter(ds2.lon,ds2.lat,c=ds2.quality_flag,vmin=0,vmax=512,edgecolor='none', cmap='jet')
ax.coastlines()
cax = plt.colorbar(cs1,shrink=.3)
ax = plt.subplot(122,projection=ccrs.PlateCarree(central_longitude=0))
cs1=ax.scatter(ds2.lon,ds2.lat,c=ds2.quality_flag,vmin=0,vmax=512,edgecolor='none', cmap='jet')
ax.coastlines()
cax = plt.colorbar(cs1,shrink=.3)
#print(ds)
fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/SSS/figures/JPLv5_r17250.png'
plt.savefig(fig_fname, transparent=False, format='png')

In [ ]:
ds

In [ ]:
ds.iqc_flag[100,1440,0].data

In [ ]:
def check_bit(flag, bit=0):
    #https://zonca.dev/2020/03/raise-check-flag-numpy.html
    return np.bitwise_and(flag, int(2**bit)) > 0
tem = check_bit(ds.iqc_flag,0)

In [ ]:
tem = check_bit(ds.iqc_flag,15)
tem[:,:,0].plot()

In [ ]:
file = 'F:/data/sat_data/smap/SSS/L2/JPL/V4.3/2018/115/SMAP_L2B_SSS_17250_20180425T004224_R16010_V4.3.h5'
ds = xr.open_dataset(file,decode_cf=False)
ds.close()
plt.rcParams['figure.figsize'] = (15.0,10.0)
ds2 = ds.where((ds.lon>-180)&(ds.smap_sss>0))
ax = plt.subplot(121,projection=ccrs.PlateCarree(central_longitude=0))
cs1=ax.scatter(ds2.lon,ds2.lat,c=ds2.smap_sss,vmin=30,vmax=36,edgecolor='none', cmap='jet')
#cs1=ax.scatter(ds2.lon,ds2.lat,c=ds2.quality_flag,vmin=0,vmax=512,edgecolor='none', cmap='jet')
ax.coastlines()
cax = plt.colorbar(cs1,shrink=.3)
ax = plt.subplot(122,projection=ccrs.PlateCarree(central_longitude=0))
cs1=ax.scatter(ds2.lon,ds2.lat,c=ds2.quality_flag,vmin=0,vmax=512,edgecolor='none', cmap='jet')
ax.coastlines()
cax = plt.colorbar(cs1,shrink=.3)
fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/SSS/figures/JPLv43_r17250.png'
plt.savefig(fig_fname, transparent=False, format='png')
#print(ds)

# What lon range for satellite & insitu? are we going 0-360 or -180 to 180?

In [ ]:
print(ds.cellon.min().data,ds.cellon.max().data)
print(ds_usv.lon.min().data,ds_usv.lon.max().data)

## First let's figure out what orbital files actually have data in our area of interest.  To do this, use the pyresample software

- read in the in situ data
- calculate the in situ min/max dates to know what files to check

Now we have our time of interest

- loop through the satellite data
- calculate the in situ min/max lat/lon on the same day to define a small box of interest
- use pyresample to map the data onto a predefined 0.1 deg resolution spatial grid
- subset the gridded map to the area of interest
- see if there is any valid data in that area
- if there is any valid data, go to next step

## Use the fast search kdtree which is part of pyresample software, but I think maybe comes originally from sci-kit-learn.

- read in the in situ data
- read in a single orbit of satellite data
- kdtree can't handle it when lat/lon are set to nan.  I frankly have no idea why there is orbital data for both the JPL and RSS products that have nan for the geolocation.  That isn't normal.  But, okay, let's deal with it.  
- stack the dataset scanline and cell positions into a new variable 'z'
- drop all variables from the dataset when the longitude is nan
- set up the tree
- loop through the orbital data
- only save a match if it is less than 0.25 deg distance AND time is less than any previous match
- save the satellite indices & some basic data onto the USV grid




In [ ]:
def get_time_start_end(isat,ds):
    if isat==0:
        orbit_time = np.datetime64(ds.attrs['time_coverage_start'])-np.timedelta64(24,'h') #changed to 24 hr for sss
        orbit_time2 = np.datetime64(ds.attrs['time_coverage_end'])+np.timedelta64(24,'h')  
    if isat==1:
        orbit_time = ds.time[0].data-np.timedelta64(12,'h')
        orbit_time2 = ds.time[-1].data+np.timedelta64(12,'h')        
    return orbit_time,orbit_time2

area_def = load_area('areas.cfg', 'pc_world')
rlon=np.arange(-180,180,.1)
rlat=np.arange(90,-90,-.1)


In [ ]:
files

In [ ]:
for iname in range(33,len(files)):  #set number of cruise to process
    
    #if ((iname<13) or (iname>15)):
    #    continue
    
    for isat in range(2):
        
        ds_usv,name_usv = read_one_usv(files[iname])

        if isat==0:
            fileout = 'F:/data/cruise_data/saildrone/sss/sss_collocations_orbital/'+name_usv+'rssv04.0_orbital.nc'
            ds_usv = add_coll_vars_ds_rss(ds_usv)
        if isat==1:
            fileout = 'F:/data/cruise_data/saildrone/sss/sss_collocations_orbital/'+name_usv+'jplv05.0_orbital.nc'   
            ds_usv = add_coll_vars_ds_jpl(ds_usv)

        #if path.exists(fileout):
        #    continue

        #search usv data
        minday,maxday = ds_usv.time[0],ds_usv.time[-1]
        usv_day = minday
        print(iname,name_usv)
        print(minday.data,maxday.data)
        while usv_day<=maxday:
            print(usv_day.data,maxday.data)
            ds_day = ds_usv.sel(time=slice(usv_day-np.timedelta64(1,'D'),usv_day+np.timedelta64(1,'D')))
            ilen = ds_day.time.size
            if ilen<1:   #don't run on days without any data
                usv_day += np.timedelta64(1,'D')
                continue
            minlon,maxlon,minlat,maxlat = ds_day.lon.min().data,ds_day.lon.max().data,ds_day.lat.min().data,ds_day.lat.max().data
            filelist = get_filelist_l2p(isat, usv_day)
            x,y,z = [],[],[]
            for ifile,file in enumerate(filelist):
                ds = xr.open_dataset(file)
                ds.close()  
                #print('****************')
                #print(file)
                if isat==0:  #change RSS data to conform with JPL definitions
                    ds = ds.isel(look=0)
                    ds = ds.rename({'iqc_flag':'quality_flag','cellon':'lon','cellat':'lat','sss_smap':'smap_sss','sss_smap_40km':'smap_sss_40km','ydim_grid':'phony_dim_0','xdim_grid':'phony_dim_1'})
                    ds['lon']=np.mod(ds.lon+180,360)-180  
                if isat==1:  #change RSS data to conform with JPL definitions
                    ds = ds.rename({'row_time':'time','ice_concentration':'fice'})

    #first do a quick check using resample to project the orbit onto a grid 
    #and quickly see if there is any data in the cruise area on that day
    #if there is, then continue to collocation

                x = ds['lon'].fillna(-89).data 
                y = ds['lat'].fillna(-89).data 
                z = ds['smap_sss'].data 
                lons,lats,data = x,y,z 
                swath_def = SwathDefinition(lons, lats)

                # Resample swath to a fixed 0.01 x 0.01 grid, represented by the variable grid_def:
                # https://stackoverflow.com/questions/58065055/floor-and-ceil-with-number-of-decimals
                #changed to be just the region of the usv cruise to make grid even smaller (hopefully)
                #when working with global orbital data, work with usv BUT
                #when working with granules use ds instead of ds_usv so you just do granule region
                grid_def_lon_min, grid_def_lon_max = np.round(ds_day.lon.min().data - 0.5 * 10**(-2), 2), np.round(ds_day.lon.max().data + 0.5 * 10**(-2), 2)
                grid_def_lat_min, grid_def_lat_max = np.round(ds_day.lat.min().data - 0.5 * 10**(-2), 2), np.round(ds_day.lat.max().data + 0.5 * 10**(-2), 2)
                grid_def_lons, grid_def_lats = np.arange(grid_def_lon_min,grid_def_lon_max+0.1,0.1), np.arange(grid_def_lat_max,grid_def_lat_min-0.1,-0.1)
                grid_mesh_lons,grid_mesh_lats = np.meshgrid(grid_def_lons,grid_def_lats)

                # Since we have the lon and lat values for the area, we define a grid instead of an area:
                # https://pyresample.readthedocs.io/en/latest/geo_def.html#griddefinition
                grid_def = GridDefinition(lons=grid_mesh_lons,lats=grid_mesh_lats)

                result1 = resample_nearest(swath_def, data, grid_def, radius_of_influence=20000, fill_value=None)
                da = xr.DataArray(result1,name='sss',coords={'lat':grid_def_lats,'lon':grid_def_lons},dims=('lat','lon'))

                numdata = np.isfinite(da).sum()
                if numdata<1:
                    continue

                #stack xarray dataset then drop lon == nan
                ds2 = ds.stack(z=('phony_dim_0', 'phony_dim_1')).reset_index('z')
                #drop nan
                ds_drop = ds2.where(np.isfinite(ds2.lon),drop=True)
                lats = ds_drop.lat.data
                lons = ds_drop.lon.data
                inputdata = list(zip(lons.ravel(), lats.ravel()))
                tree = spatial.KDTree(inputdata)

                orbit_time, orbit_time2 = get_time_start_end(isat,ds)

                cond = (ds_usv.time.data>orbit_time) & (ds_usv.time.data<orbit_time2)
                item = np.argwhere(cond)
                if item.sum()<1:  #no data within 12 hr of orbit
                    continue
                for iusv_index in range(int(item[0]),int(item[-1])):
                    pts = np.array([ds_usv.lon[iusv_index], ds_usv.lat[iusv_index]]) #pts = np.array([ds_usv.lon[iusv]+360
                    tree.query(pts,k=1)
                    i = tree.query(pts)[1]
                    rdist = tree.query(pts)[0]                   
                    if rdist>.25:    #don't use matchups more than 25 km away
                        continue
                    #use .where to find the original indices of the matched data point
                    #find by matching sss and lat, just randomly chosen variables, you could use any
                    result = np.where((ds.smap_sss == ds_drop.smap_sss[i].data) & (ds.lat == ds_drop.lat[i].data))
                    listOfCoordinates = list(zip(result[0], result[1]))
                    if len(listOfCoordinates)==0:
                        continue
                    ii, jj = listOfCoordinates[0][0],listOfCoordinates[0][1]
                    if isat==0:
                        deltaTa = ((ds_usv.time[iusv_index]-ds.time[ii,jj]).data)/ np.timedelta64(1,'m')
                    if isat==1:
                        deltaTa = ((ds_usv.time[iusv_index]-ds.time[ii]).data)/ np.timedelta64(1,'m')
                    if np.abs(deltaTa)<np.abs(ds_usv.deltaT[iusv_index].data):
                        ds_usv.deltaT[iusv_index]=deltaTa
                        ds_usv.smap_SSS[iusv_index]=ds.smap_sss[ii,jj]
                        if isat==0:
                            ds_usv.smap_SSS_40km[iusv_index]=ds.smap_sss_40km[ii,jj]
                            ds_usv.smap_fland[iusv_index]=ds.fland[ii,jj]
                            ds_usv.smap_rev_number[iusv_index]=int(ds.attrs['orbit_number'])
                        else:
                            ds_usv.smap_rev_number[iusv_index]=int(ds.attrs['REVNO'])
                        ds_usv.smap_iqc_flag[iusv_index]=ds.quality_flag[ii,jj].astype('int') #test int
                        ds_usv.smap_name[iusv_index]=str(file)
                        ds_usv.smap_fice[iusv_index]=ds.fice[ii,jj]
                        ds_usv.smap_dist[iusv_index]=rdist
                        ds_usv.smap_ydim[iusv_index]=ii
                        ds_usv.smap_xdim[iusv_index]=jj
            usv_day += np.timedelta64(1,'D')

        #add distance to land
        ds_usv['dist_land']=ds_land.dist_land.interp(lat=ds_usv.lat,lon=ds_usv.lon).drop({'lat','lon'})
        lnd_att={'long_name':'distance to nearest land','units':'km'}
        ds_usv['dist_land'].attrs=lnd_att    

        ds_usv.deltaT.attrs={'long_name':'time difference between saildrone and satellite','units':'minutes'}
        ds_usv.smap_SSS.attrs=ds.smap_sss.attrs
        if isat==0:
            ds_usv.smap_SSS_40km.attrs=ds.smap_sss_40km.attrs
            ds_usv.smap_fland.attrs=ds.fland.attrs
        ds_usv.smap_iqc_flag.attrs=ds.quality_flag.attrs
        ds_usv.smap_fice.attrs=ds.fice.attrs


        ds_usv.to_netcdf(fileout)


In [ ]:
for iname in range(1):  #set number of cruise to process
    for isat in range(1):
        ds_usv,name_usv = read_one_usv(files[iname])
        ds_usv = add_coll_vars_ds_rss(ds_usv)


In [ ]:
ds_usv.smap_iqc_flag.min()

In [ ]:
ds.quality_flag[ii,jj].astype('int')

In [ ]:
#test collocation
import os
from glob import glob
import xarray as xr
dir_data_pattern = 'F:/data/cruise_data/saildrone/sss/sss_collocations_orbital/*.nc' 
files = glob(dir_data_pattern)
ds = xr.open_dataset(files[0])
ds.close
ds1 = xr.open_dataset(files[1])
ds1.close
ds

In [ ]:
ds.smap_iqc_flag.plot()
(ds1.smap_iqc_flag/36000*500).plot()

In [ ]:
ds.smap_SSS.plot()
ds.SAL_CTD_MEAN.plot()
ds1.smap_SSS.plot()

In [ ]:
isat=0
file = 'F:/data/sat_data/smap/SSS/L2/RSS/V4/SCI/2018/115/RSS_SMAP_SSS_L2C_r17250_20180425T004136_2018115_FNL_V04.0.nc'
ds = xr.open_dataset(file,decode_cf=False)
if isat==0:  #change RSS data to conform with JPL definitions
    ds = ds.isel(look=0)
    ds = ds.rename({'iqc_flag':'quality_flag','cellon':'lon','cellat':'lat','sss_smap':'smap_sss','sss_smap_40km':'smap_sss_40km','ydim_grid':'phony_dim_0','xdim_grid':'phony_dim_1'})
    ds['lon']=np.mod(ds.lon+180,360)-180  
if isat==1:  #change RSS data to conform with JPL definitions
    ds = ds.rename({'row_time':'time','ice_concentration':'fice'})
ds_usv = xr.open_dataset('F:/data/cruise_data/saildrone/sss/old/sss_collocations_orbital/saildrone_tpos_sd1005_2017jplv04.3_orbital.nc')
#ds_usv = xr.open_dataset('F:/data/cruise_data/saildrone/sss/old/sss_collocations_orbital/saildrone_tpos_sd1005_2017rssv04.0_orbital.nc')
ds2=ds_usv
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0))
cs1=ax.scatter(ds2.lon,ds2.lat,c=ds2.smap_iqc_flag,vmin=0,vmax=512,edgecolor='none', cmap='jet')
ax.coastlines()
cax = plt.colorbar(cs1)